# Data Jointure & Explose - Eplore data

## Configuration

In [1]:
%AddJar file:///home/jovyan/work/apps/Emiasd-Flight-Data-Analysis.jar

Starting download from file:///home/jovyan/work/apps/Emiasd-Flight-Data-Analysis.jar
Finished download of Emiasd-Flight-Data-Analysis.jar
Using cached version of Emiasd-Flight-Data-Analysis.jar


In [2]:
import org.apache.spark.sql.SparkSession
import com.flightdelay.config.{AppConfiguration, ConfigurationLoader, ExperimentConfig}
import com.flightdelay.data.loaders.FlightDataLoader

// Env Configuration
val args: Array[String] = Array("jupyter")
implicit val configuration: AppConfiguration = ConfigurationLoader.loadConfiguration(args)
implicit val experiment: ExperimentConfig = configuration.experiments(0)

val spark = SparkSession.builder()
  .config(sc.getConf)
  .config("spark.eventLog.enabled", "true")
  .config("spark.eventLog.dir", "file:///home/jovyan/work/spark-events")
  .getOrCreate()

// Rendre la session Spark implicite
implicit val session = spark



args = Array(jupyter)
configuration = AppConfiguration(local,CommonConfig(42,true,debug,false,false,DataConfig(/home/jovyan/work/data,FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Flights/201201*.csv),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Weather/20101*.txt),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/wban_airport_timezone.csv)),OutputConfig(/home/jovyan/work/output,FileConfig(/home/jovyan/work/output/data),FileConfig(/home/jovyan/work/output/model)),MLFlowConfig(false,http://localhost:5555)),Stream(ExperimentConfig(Experience-local,Baseline Random ...


AppConfiguration(local,CommonConfig(42,true,debug,false,false,DataConfig(/home/jovyan/work/data,FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Flights/201201*.csv),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Weather/20101*.txt),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/wban_airport_timezone.csv)),OutputConfig(/home/jovyan/work/output,FileConfig(/home/jovyan/work/output/data),FileConfig(/home/jovyan/work/output/model)),MLFlowConfig(false,http://localhost:5555)),Stream(ExperimentConfig(Experience-local,Baseline Random ...

## Chargement des données

In [3]:
val trainPath = s"${configuration.common.output.basePath}/${experiment.name}/data/join_exploded_train_prepared.parquet"
val trainDF = spark.read.parquet(trainPath)

println("Train DF Count: ", trainDF.count())

(Train DF Count: ,2675)


trainPath = /home/jovyan/work/output/Experience-local/data/join_exploded_train_prepared.parquet
trainDF = [CRS_DEP_TIME: int, CRS_ELAPSED_TIME: double ... 72 more fields]


[CRS_DEP_TIME: int, CRS_ELAPSED_TIME: double ... 72 more fields]

In [4]:
val testPath = s"${configuration.common.output.basePath}/${experiment.name}/data/join_exploded_test_prepared.parquet"
val testDF = spark.read.parquet(testPath)

println("Test DF Count: ", testDF.count())

(Test DF Count: ,916)


testPath = /home/jovyan/work/output/Experience-local/data/join_exploded_test_prepared.parquet
testDF = [CRS_DEP_TIME: int, CRS_ELAPSED_TIME: double ... 72 more fields]


[CRS_DEP_TIME: int, CRS_ELAPSED_TIME: double ... 72 more fields]

In [5]:
trainDF.printSchema

root
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- DEST_AIRPORT_ID: integer (nullable = true)
 |-- DEST_WBAN: string (nullable = true)
 |-- OP_CARRIER_AIRLINE_ID: integer (nullable = true)
 |-- ORIGIN_AIRPORT_ID: integer (nullable = true)
 |-- ORIGIN_WBAN: string (nullable = true)
 |-- UTC_FL_DATE: date (nullable = true)
 |-- feature_arrival_time_period: string (nullable = true)
 |-- feature_departure_hour_rounded: long (nullable = true)
 |-- feature_departure_time_period: string (nullable = true)
 |-- feature_flight_week_of_year: integer (nullable = true)
 |-- feature_utc_departure_hour_rounded: long (nullable = true)
 |-- is_delayed: integer (nullable = true)
 |-- origin_weather_feature_pressure_bucket-1: string (nullable = true)
 |-- origin_weather_feature_flight_category-1: string (nullable = true)
 |-- origin_weather_weather_hazard_level-1: integer (nullable = true)
 |-- origin_weather_feature_weather_severity_index-1: double (nu

In [8]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

val df = trainDF

// df : ton DataFrame
val total = df.count()

// Colonnes qu'on considère comme catégorielles
val categoricalCols: Array[String] =
  df.schema.fields.collect {
    case StructField(name, StringType, _, _)  => name
    case StructField(name, IntegerType, _, _) => name
  }

println("=== Colonnes catégorielles détectées ===")
categoricalCols.foreach(println)

// Stats par colonne catégorielle
categoricalCols.foreach { colName =>
  val distinctCount = df.select(col(colName)).distinct().count()
  val nullCount = df.filter(col(colName).isNull || col(colName) === "").count()
  val nullPct = if (total == 0) 0.0 else nullCount.toDouble * 100.0 / total

  println(s"\n----- $colName -----")
  println(s"Total lignes      : $total")
  println(s"Nulls             : $nullCount")
  println(s"% de Nulls        : ${"%.2f".format(nullPct)} %")
  println(s"Nb de catégories  : $distinctCount")
}

=== Colonnes catégorielles détectées ===
CRS_DEP_TIME
DEST_AIRPORT_ID
DEST_WBAN
OP_CARRIER_AIRLINE_ID
ORIGIN_AIRPORT_ID
ORIGIN_WBAN
feature_arrival_time_period
feature_departure_time_period
feature_flight_week_of_year
is_delayed
origin_weather_feature_pressure_bucket-1
origin_weather_feature_flight_category-1
origin_weather_weather_hazard_level-1
origin_weather_feature_visibility_category-1
origin_weather_feature_flight_category_ordinal-1
origin_weather_feature_requires_cat_ii-1
origin_weather_extracted_codes-1
origin_weather_feature_operations_risk_level-1
origin_weather_feature_most_critical_sky-1
origin_weather_feature_is_very_low_visibility-1
origin_weather_feature_pressure_bucket-0
origin_weather_feature_flight_category-0
origin_weather_weather_hazard_level-0
origin_weather_feature_visibility_category-0
origin_weather_feature_flight_category_ordinal-0
origin_weather_feature_requires_cat_ii-0
origin_weather_extracted_codes-0
origin_weather_feature_operations_risk_level-0
origin_wea

df = [CRS_DEP_TIME: int, CRS_ELAPSED_TIME: double ... 72 more fields]
total = 2675
categoricalCols = Array(CRS_DEP_TIME, DEST_AIRPORT_ID, DEST_WBAN, OP_CARRIER_AIRLINE_ID, ORIGIN_AIRPORT_ID, ORIGIN_WBAN, feature_arrival_time_period, feature_departure_time_period, feature_flight_week_of_year, is_delayed, origin_weather_feature_pressure_bucket-1, origin_weather_feature_flight_category-1, origin_weather_weather_hazard_level-1, origin_weather_feature_visibility_category-1, origin_weather_feature_flight_category_ordinal-1, origin_weather_feature_requires_cat_ii-1, origin_weather_extracted_codes-1, origin_weather_feature_operations_risk_level-1, origin_weather_featu...


Array(CRS_DEP_TIME, DEST_AIRPORT_ID, DEST_WBAN, OP_CARRIER_AIRLINE_ID, ORIGIN_AIRPORT_ID, ORIGIN_WBAN, feature_arrival_time_period, feature_departure_time_period, feature_flight_week_of_year, is_delayed, origin_weather_feature_pressure_bucket-1, origin_weather_feature_flight_category-1, origin_weather_weather_hazard_level-1, origin_weather_feature_visibility_category-1, origin_weather_feature_flight_category_ordinal-1, origin_weather_feature_requires_cat_ii-1, origin_weather_extracted_codes-1, origin_weather_feature_operations_risk_level-1, origin_weather_featu...

In [13]:
trainDF.filter(col("destination_weather_feature_is_very_low_visibility-0").isNull).show(10,10000, true)

-RECORD 0------------------------------------------------------------------
 CRS_DEP_TIME                                            | 1620            
 CRS_ELAPSED_TIME                                        | 54.0            
 DEST_AIRPORT_ID                                         | 13577           
 DEST_WBAN                                               | 13717           
 OP_CARRIER_AIRLINE_ID                                   | 20378           
 ORIGIN_AIRPORT_ID                                       | 11057           
 ORIGIN_WBAN                                             | 13881           
 UTC_FL_DATE                                             | 2012-07-20      
 feature_arrival_time_period                             | Late_Afternoon  
 feature_departure_hour_rounded                          | 1600            
 feature_departure_time_period                           | Late_Afternoon  
 feature_flight_week_of_year                             | 29              
 feature_utc